## Examples of degirum_tools.streams module usage
This notebook contains various examples, which demonstrate the capabilities of degirum_tools.streams module: streaming toolkit for PySDK.

The script also uses a video file to run this code. The video file path needs to be specified by defining video_source variable.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

In [ ]:
# video_source: video source for inference
#     camera index for local camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
video_source = (
    "https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/example_video.mp4"
)

### Simplest example: video source is connected to video display

In [ ]:
from degirum_tools.streams import *

# create Composition: an object, which keeps all streaming blocks (called gizmos) and runs them in parallel
c = Composition()

# create gizmos:
source = VideoSourceGizmo(video_source)  # video source gizmo
display = VideoDisplayGizmo(
    "press `x` or `q` to stop", allow_drop=False
)  # video display gizmo

# Create pipeline: connect display input to source output
display.connect_to(source)

# add gizmos to composition
c.add(source)
c.add(display)

# start composition
c.start()

# call c.stop() to stop composition or just press `x` or `q` in display window

### Same example, but with compact syntax

In [ ]:
from degirum_tools.streams import *

c = Composition()

# Create gizmos and pipeline as a single-liner:
# we use __call__() operator of Composition class instead of add() method
# and we use `>>` operator of gizmo classes instead of connect_to() method
c(VideoSourceGizmo(video_source)) >> c(VideoDisplayGizmo())

c.start()

### Example of forked streams

In [ ]:
from degirum_tools.streams import *

c = Composition()

# create and add to composition all required gizmos
source = c.add(VideoSourceGizmo(video_source))  # video source gizmo
display = c.add(
    VideoDisplayGizmo(["Original", "Resized"])
)  # two-input display gizmo: will show two windows
resizer = c.add(ResizingGizmo(300, 200))  # resizing gizmo

# Create pipeline: the image source is fed to a display and to the image resizing gizmo,
# which is then fed to another display.

display.connect_to(source, 0)  # display input 0 is "Original"
resizer.connect_to(source)
display.connect_to(resizer, 1)  # display input 1 is "Resized"

c.start()

### Example of capturing video input into file with simultaneous display on a screen

In [ ]:
from degirum_tools.streams import *

c = Composition()

source = c.add(VideoSourceGizmo(video_source))
display = c.add(VideoDisplayGizmo())
saver = c.add(VideoSaverGizmo("temp/mycapture.mp4"))

source >> display
source >> saver

c.start()

### Example of AI inference from the camera with AI inference results display

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option. 

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

In [ ]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_name: name of the model for running AI inference
hw_location = "@cloud"
model_zoo_url = "https://cs.degirum.com/degirum/public"
model_name = "yolo_v5s_coco--512x512_quant_n2x_orca1_1"

In [ ]:
import degirum as dg, degirum_tools
from degirum_tools.streams import *

# connect to AI inference engine
zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token())

# load some object detection AI model
model = zoo.load_model(model_name)

c = Composition()

# create gizmos
source = c.add(VideoSourceGizmo(video_source))  # video source
detection = c.add(AiSimpleGizmo(model))  # AI model
display = c.add(
    VideoDisplayGizmo("Detection", show_ai_overlay=True, show_fps=True)
)  # display

# create pipeline
source >> detection >> display

c.start()

### Example of AI inference with separate pre-processing stage

Frames from the camera are supplied to the object detection model preprocessor and then to the object detection model
itself. This improves performance by allowing the image resizing preprocessor to run in a separate thread.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option. 

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

In [ ]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_name: name of the model for running AI inference
hw_location = "@cloud"
model_zoo_url = "https://cs.degirum.com/degirum/public"
model_name = "yolo_v5s_coco--512x512_quant_n2x_orca1_1"

In [ ]:
import degirum as dg, degirum_tools
from degirum_tools.streams import *

# connect to AI inference engine
zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token())

# load AI model
model = zoo.load_model(model_name)

c = Composition()

# create gizmos
source = c.add(VideoSourceGizmo(video_source))  # video source
preprocessor = c.add(AiPreprocessGizmo(model))
detection = c.add(AiSimpleGizmo(model))
display = c.add(
    VideoDisplayGizmo("Objects", show_ai_overlay=True, show_fps=True)
)  # display

# create pipeline
source >> preprocessor >> detection >> display

c.start()